In [1]:
import torch
from mmdet.apis import inference_detector, init_detector
from mmdet.models import build_detector
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmcv.runner import load_checkpoint
import os


# Pseudo labeling을 위해 필요한 정보 설정
config_file = '/opt/ml/level2_objectdetection-cv-12/mmdetection/configs/_cascadeRCNN_swinL_fold4/final.py'  # 학습에 사용된 모델의 구성 파일
cfg = Config.fromfile(config_file)#faster_rcnn_x101_32x4d_fpn_1x_coco

checkpoint_path = '/opt/ml/level2_objectdetection-cv-12/mmdetection/work_dirs/swin_base_fold4_v2/best_bbox_mAP_50_epoch_11.pth'  # 학습된 모델의 체크포인트 파일
data_root = '/opt/ml/dataset'  # COCO 데이터셋의 루트 디렉토리

# COCO 데이터셋의 클래스 이름
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")



/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [2]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

model = init_detector(config_file, checkpoint_path, device='cuda:0')
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
load checkpoint from local path: /opt/ml/level2_objectdetection-cv-12/mmdetection/work_dirs/swin_base_fold4_v2/best_bbox_mAP_50_epoch_11.pth


/opt/ml/level2_objectdetection-cv-12/mmdetection/mmdet/apis/inference.py:51: UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


load checkpoint from local path: /opt/ml/level2_objectdetection-cv-12/mmdetection/work_dirs/swin_base_fold4_v2/best_bbox_mAP_50_epoch_11.pth


In [4]:

from pycocotools.coco import COCO
import json

annotation = '/opt/ml/level2_objectdetection-cv-12/mmdetection/inference_json/test_addbbox.json'
with open(annotation) as f: data = json.load(f)
image_ids = []
class_name = []
class_id = []
x_min = []
y_min = []
x_max = []
y_max = []
classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

train_annos = []
bbox_id = 0

annotations = data['annotations']

for ann in annotations : 
    # if ann['score'] < 0.5:
        # continue
    ann['id'] = bbox_id
    x, y, w, h = ann['bbox'] 
    x = round(x, 1)
    y = round(y, 1)
    w = round(w, 1)
    h = round(h, 1)
    bbox_ = [x, y, w, h]
    ann['bbox'] = bbox_
    ann['area'] = w*h
    ann['score'] = round(ann['score'], 3)

    train_annos.append(ann)

    bbox_id+=1

train_data = {
    'info' : data['info'],
    'licenses' : data['licenses'],
    'images' : data['images'],
    'categories' : data['categories'],
    'annotations' : train_annos
                    }
with open(f'/opt/ml/level2_objectdetection-cv-12/mmdetection/inference_json/swinB_fold4.bbox_addid.json', 'w') as f:
    json.dump(train_data, f, indent=4)
print('completed')


completed
